# NOTICE

Before running the code, please read the Readme.MD file bundled with this Notebook for information on what the goal of this project is.

# Step 1 - Dumping Data Using Python

This notebook is written to query Wikipedia's PageViews API and retrieve information for each article in an external list, intended to contain the names of the articles for movies that have won an Academy Award. It takes in article names from the 'name' column of a CSV in the same directory as this notebook, named "thank\_the\_academy.AUG.2023.csv", and queries the PageView API to retrieve the data for all views for those articles with the 'desktop', 'mobile-web', and 'mobile-app' access categories; in other words, it retrieves how many people saw each corresponding article from their desktop, or mobile device using either the browser or the Wikipedia app. The code is written to grab data for each month from January 1st, 2015 to April 1st, 2023. The response for each API call is then stored verbatim in an individual JSON file for further compilation in other parts of this assignment. For information on the entire assignment, please refer to the README.md file in the same folder as this notebook.

## Setup

This next three code cells import the packages required to query and dump the API data, the basic constants that will be used, and the functions that will be used to iteratively query the API. Most of the code in these three cells was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program, and made available under the [Creative a Commons](https://creativecommons.org/) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.2 - August 14, 2023 . Minor changes and additions were made to also query the API for mobile-web and mobile-app view data, as well as dump the data retrieved into individual JSON files.

In [1]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

In [3]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<uwnetid@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = ['Northern flicker', 'Bison', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# These templates are used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. F
ARTICLE_PAGEVIEWS_PARAMS_DESKTOP = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this is changed for the different access types (see below)
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015010100",   # start and end dates need to be set
    "end":         "2023040100"    # this is likely the wrong end date
}

ARTICLE_PAGEVIEWS_PARAMS_MOBILE_APP = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-app",     
    "agent":       "user",
    "article":     "",
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2023040100"
}

ARTICLE_PAGEVIEWS_PARAMS_MOBILE_WEB = {
    "project":     "en.wikipedia.org",
    "access":      "mobile-web",
    "agent":       "user",
    "article":     "",
    "granularity": "monthly",
    "start":       "2015010100",
    "end":         "2023040100"
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [6]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_MOBILE_WEB,
                                  headers = REQUEST_HEADERS):
    

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

def dump_to_file(view_data, title, folder):

    #forbidden characters
    title = title.replace("\\", "_")
    title = title.replace(":", "_")
    title = title.replace("/", "_")
    title = title.replace("*", "_")
    title = title.replace('"', "_")
    title = title.replace("<", "_")
    title = title.replace(">", "_")
    title = title.replace("?", "_")


    print("writing pageview data file for: ", title)

    with open((folder + "/" + title + ".json"), "w") as outfile:
        json.dump(view_data, outfile)

def get_pageView_data_with_template(template, base_folder, article_title_list):
    for article in article_title_list:
        accessType = template["access"]
        print("Getting " + accessType + " pageview data for: ", article)
        data = request_pageviews_per_article(article, request_template = template)
        dump_to_file(data, article, (base_folder + "/" + accessType))



## Querying API

The below four cells take in the list of articles from the CSV with the specifications listed in the header, and use it to get desktop, mobile-web, and mobile-app views from the PageView API, and dump them to individual JSON files in a subdirectory.

In [7]:
#getting article list
import pandas as pd 
article_csv = pd.read_csv("thank_the_academy.AUG.2023.csv")
article_list = article_csv['name'].tolist()

In [11]:
get_pageView_data_with_template(ARTICLE_PAGEVIEWS_PARAMS_MOBILE_WEB, "raw_data", article_list)

Getting mobile-web pageview data for:  Everything Everywhere All at Once
writing pageview data file for:  Everything Everywhere All at Once
Getting mobile-web pageview data for:  All Quiet on the Western Front (2022 film)
writing pageview data file for:  All Quiet on the Western Front (2022 film)
Getting mobile-web pageview data for:  The Whale (2022 film)
writing pageview data file for:  The Whale (2022 film)
Getting mobile-web pageview data for:  Top Gun: Maverick
writing pageview data file for:  Top Gun_ Maverick
Getting mobile-web pageview data for:  Black Panther: Wakanda Forever
writing pageview data file for:  Black Panther_ Wakanda Forever
Getting mobile-web pageview data for:  Avatar: The Way of Water
writing pageview data file for:  Avatar_ The Way of Water
Getting mobile-web pageview data for:  Women Talking (film)
writing pageview data file for:  Women Talking (film)
Getting mobile-web pageview data for:  Guillermo del Toro's Pinocchio
writing pageview data file for:  Guill

writing pageview data file for:  Skin (2018 short film)
Getting mobile-web pageview data for:  Spider-Man: Into the Spider-Verse
writing pageview data file for:  Spider-Man_ Into the Spider-Verse
Getting mobile-web pageview data for:  The Shape of Water
writing pageview data file for:  The Shape of Water
Getting mobile-web pageview data for:  Dunkirk (2017 film)
writing pageview data file for:  Dunkirk (2017 film)
Getting mobile-web pageview data for:  Three Billboards Outside Ebbing, Missouri
writing pageview data file for:  Three Billboards Outside Ebbing, Missouri
Getting mobile-web pageview data for:  Darkest Hour (film)
writing pageview data file for:  Darkest Hour (film)
Getting mobile-web pageview data for:  Blade Runner 2049
writing pageview data file for:  Blade Runner 2049
Getting mobile-web pageview data for:  Coco (2017 film)
writing pageview data file for:  Coco (2017 film)
Getting mobile-web pageview data for:  Phantom Thread
writing pageview data file for:  Phantom Threa

writing pageview data file for:  The Lady in Number 6
Getting mobile-web pageview data for:  Helium (film)
writing pageview data file for:  Helium (film)
Getting mobile-web pageview data for:  The Great Beauty
writing pageview data file for:  The Great Beauty
Getting mobile-web pageview data for:  20 Feet from Stardom
writing pageview data file for:  20 Feet from Stardom
Getting mobile-web pageview data for:  Argo (2012 film)
writing pageview data file for:  Argo (2012 film)
Getting mobile-web pageview data for:  Life of Pi (film)
writing pageview data file for:  Life of Pi (film)
Getting mobile-web pageview data for:  Les Misérables (2012 film)
writing pageview data file for:  Les Misérables (2012 film)
Getting mobile-web pageview data for:  Lincoln (film)
writing pageview data file for:  Lincoln (film)
Getting mobile-web pageview data for:  Django Unchained
writing pageview data file for:  Django Unchained
Getting mobile-web pageview data for:  Skyfall
writing pageview data file for:

writing pageview data file for:  The Bourne Ultimatum (film)
Getting mobile-web pageview data for:  There Will Be Blood
writing pageview data file for:  There Will Be Blood
Getting mobile-web pageview data for:  La Vie en rose (film)
writing pageview data file for:  La Vie en rose (film)
Getting mobile-web pageview data for:  Atonement (2007 film)
writing pageview data file for:  Atonement (2007 film)
Getting mobile-web pageview data for:  Michael Clayton (film)
writing pageview data file for:  Michael Clayton (film)
Getting mobile-web pageview data for:  Ratatouille (film)
writing pageview data file for:  Ratatouille (film)
Getting mobile-web pageview data for:  Juno (film)
writing pageview data file for:  Juno (film)
Getting mobile-web pageview data for:  Sweeney Todd: The Demon Barber of Fleet Street (2007 film)
writing pageview data file for:  Sweeney Todd_ The Demon Barber of Fleet Street (2007 film)
Getting mobile-web pageview data for:  The Golden Compass (film)
writing pageview

writing pageview data file for:  Finding Nemo
Getting mobile-web pageview data for:  The Barbarian Invasions
writing pageview data file for:  The Barbarian Invasions
Getting mobile-web pageview data for:  Two Soldiers (2003 film)
writing pageview data file for:  Two Soldiers (2003 film)
Getting mobile-web pageview data for:  Monster (2003 film)
writing pageview data file for:  Monster (2003 film)
Getting mobile-web pageview data for:  Harvie Krumpet
writing pageview data file for:  Harvie Krumpet
Getting mobile-web pageview data for:  Chernobyl Heart
writing pageview data file for:  Chernobyl Heart
Getting mobile-web pageview data for:  The Fog of War
writing pageview data file for:  The Fog of War
Getting mobile-web pageview data for:  Chicago (2002 film)
writing pageview data file for:  Chicago (2002 film)
Getting mobile-web pageview data for:  The Pianist (2002 film)
writing pageview data file for:  The Pianist (2002 film)
Getting mobile-web pageview data for:  The Lord of the Rings

...

writing pageview data file for:  Kukan
Getting mobile-web pageview data for:  La Strada
writing pageview data file for:  La Strada
Getting mobile-web pageview data for:  La Dolce Vita
writing pageview data file for:  La Dolce Vita
Getting mobile-web pageview data for:  Lady Be Good (1941 film)
writing pageview data file for:  Lady Be Good (1941 film)
Getting mobile-web pageview data for:  The Last Command (1928 film)
writing pageview data file for:  The Last Command (1928 film)
Getting mobile-web pageview data for:  Laura (1944 film)
writing pageview data file for:  Laura (1944 film)
Getting mobile-web pageview data for:  The Lavender Hill Mob
writing pageview data file for:  The Lavender Hill Mob
Getting mobile-web pageview data for:  Lawrence of Arabia (film)
writing pageview data file for:  Lawrence of Arabia (film)
Getting mobile-web pageview data for:  Leave Her to Heaven
writing pageview data file for:  Leave Her to Heaven
Getting mobile-web pageview data for:  Lend a Paw
writing

writing pageview data file for:  Nights of Cabiria
Getting mobile-web pageview data for:  Nine from Little Rock
writing pageview data file for:  Nine from Little Rock
Getting mobile-web pageview data for:  None but the Lonely Heart (film)
writing pageview data file for:  None but the Lonely Heart (film)
Getting mobile-web pageview data for:  North West Mounted Police (film)
writing pageview data file for:  North West Mounted Police (film)
Getting mobile-web pageview data for:  Now, Voyager
writing pageview data file for:  Now, Voyager
Getting mobile-web pageview data for:  An Occurrence at Owl Creek Bridge (film)
writing pageview data file for:  An Occurrence at Owl Creek Bridge (film)
Getting mobile-web pageview data for:  Of Pups and Puzzles
writing pageview data file for:  Of Pups and Puzzles
Getting mobile-web pageview data for:  Oklahoma! (1955 film)
writing pageview data file for:  Oklahoma! (1955 film)
Getting mobile-web pageview data for:  The Old Man and the Sea (1958 film)
wr

writing pageview data file for:  Serengeti Shall Not Die
Getting mobile-web pageview data for:  Sergeant York (film)
writing pageview data file for:  Sergeant York (film)
Getting mobile-web pageview data for:  Seven Brides for Seven Brothers
writing pageview data file for:  Seven Brides for Seven Brothers
Getting mobile-web pageview data for:  Seven Days to Noon
writing pageview data file for:  Seven Days to Noon
Getting mobile-web pageview data for:  The Seventh Veil
writing pageview data file for:  The Seventh Veil
Getting mobile-web pageview data for:  Shane (film)
writing pageview data file for:  Shane (film)
Getting mobile-web pageview data for:  Shanghai Express (film)
writing pageview data file for:  Shanghai Express (film)
Getting mobile-web pageview data for:  She Wore a Yellow Ribbon
writing pageview data file for:  She Wore a Yellow Ribbon
Getting mobile-web pageview data for:  Shoeshine (film)
writing pageview data file for:  Shoeshine (film)
Getting mobile-web pageview dat

writing pageview data file for:  Three Little Pigs (film)
Getting mobile-web pageview data for:  Three Orphan Kittens
writing pageview data file for:  Three Orphan Kittens
Getting mobile-web pageview data for:  Through a Glass Darkly (film)
writing pageview data file for:  Through a Glass Darkly (film)
Getting mobile-web pageview data for:  Thunderball (film)
writing pageview data file for:  Thunderball (film)
Getting mobile-web pageview data for:  Thursday's Children
writing pageview data file for:  Thursday's Children
Getting mobile-web pageview data for:  The Time Machine (1960 film)
writing pageview data file for:  The Time Machine (1960 film)
Getting mobile-web pageview data for:  A Time Out of War
writing pageview data file for:  A Time Out of War
Getting mobile-web pageview data for:  Tin Pan Alley (film)
writing pageview data file for:  Tin Pan Alley (film)
Getting mobile-web pageview data for:  The Titan: Story of Michelangelo
writing pageview data file for:  The Titan_ Story 

writing pageview data file for:  Wuthering Heights (1939 film)
Getting mobile-web pageview data for:  Yankee Doodle Dandy
writing pageview data file for:  Yankee Doodle Dandy
Getting mobile-web pageview data for:  The Yankee Doodle Mouse
writing pageview data file for:  The Yankee Doodle Mouse
Getting mobile-web pageview data for:  The Yearling (1946 film)
writing pageview data file for:  The Yearling (1946 film)
Getting mobile-web pageview data for:  Yesterday, Today and Tomorrow
writing pageview data file for:  Yesterday, Today and Tomorrow
Getting mobile-web pageview data for:  You Can't Take It with You (film)
writing pageview data file for:  You Can't Take It with You (film)
Getting mobile-web pageview data for:  Zorba the Greek (film)
writing pageview data file for:  Zorba the Greek (film)


In [10]:
get_pageView_data_with_template(ARTICLE_PAGEVIEWS_PARAMS_DESKTOP, "raw_data", article_list)

Getting desktop pageview data for:  Everything Everywhere All at Once
writing pageview data file for:  Everything Everywhere All at Once
Getting desktop pageview data for:  All Quiet on the Western Front (2022 film)
writing pageview data file for:  All Quiet on the Western Front (2022 film)
Getting desktop pageview data for:  The Whale (2022 film)
writing pageview data file for:  The Whale (2022 film)
Getting desktop pageview data for:  Top Gun: Maverick
writing pageview data file for:  Top Gun_ Maverick
Getting desktop pageview data for:  Black Panther: Wakanda Forever
writing pageview data file for:  Black Panther_ Wakanda Forever
Getting desktop pageview data for:  Avatar: The Way of Water
writing pageview data file for:  Avatar_ The Way of Water
Getting desktop pageview data for:  Women Talking (film)
writing pageview data file for:  Women Talking (film)
Getting desktop pageview data for:  Guillermo del Toro's Pinocchio
writing pageview data file for:  Guillermo del Toro's Pinocchi

writing pageview data file for:  Spider-Man_ Into the Spider-Verse
Getting desktop pageview data for:  The Shape of Water
writing pageview data file for:  The Shape of Water
Getting desktop pageview data for:  Dunkirk (2017 film)
writing pageview data file for:  Dunkirk (2017 film)
Getting desktop pageview data for:  Three Billboards Outside Ebbing, Missouri
writing pageview data file for:  Three Billboards Outside Ebbing, Missouri
Getting desktop pageview data for:  Darkest Hour (film)
writing pageview data file for:  Darkest Hour (film)
Getting desktop pageview data for:  Blade Runner 2049
writing pageview data file for:  Blade Runner 2049
Getting desktop pageview data for:  Coco (2017 film)
writing pageview data file for:  Coco (2017 film)
Getting desktop pageview data for:  Phantom Thread
writing pageview data file for:  Phantom Thread
Getting desktop pageview data for:  Call Me by Your Name (film)
writing pageview data file for:  Call Me by Your Name (film)
Getting desktop pagevie

writing pageview data file for:  20 Feet from Stardom
Getting desktop pageview data for:  Argo (2012 film)
writing pageview data file for:  Argo (2012 film)
Getting desktop pageview data for:  Life of Pi (film)
writing pageview data file for:  Life of Pi (film)
Getting desktop pageview data for:  Les Misérables (2012 film)
writing pageview data file for:  Les Misérables (2012 film)
Getting desktop pageview data for:  Lincoln (film)
writing pageview data file for:  Lincoln (film)
Getting desktop pageview data for:  Django Unchained
writing pageview data file for:  Django Unchained
Getting desktop pageview data for:  Skyfall
writing pageview data file for:  Skyfall
Getting desktop pageview data for:  Silver Linings Playbook
writing pageview data file for:  Silver Linings Playbook
Getting desktop pageview data for:  Zero Dark Thirty
writing pageview data file for:  Zero Dark Thirty
Getting desktop pageview data for:  Amour (2012 film)
writing pageview data file for:  Amour (2012 film)
Get

writing pageview data file for:  Michael Clayton (film)
Getting desktop pageview data for:  Ratatouille (film)
writing pageview data file for:  Ratatouille (film)
Getting desktop pageview data for:  Juno (film)
writing pageview data file for:  Juno (film)
Getting desktop pageview data for:  Sweeney Todd: The Demon Barber of Fleet Street (2007 film)
writing pageview data file for:  Sweeney Todd_ The Demon Barber of Fleet Street (2007 film)
Getting desktop pageview data for:  The Golden Compass (film)
writing pageview data file for:  The Golden Compass (film)
Getting desktop pageview data for:  Elizabeth: The Golden Age
writing pageview data file for:  Elizabeth_ The Golden Age
Getting desktop pageview data for:  Taxi to the Dark Side
writing pageview data file for:  Taxi to the Dark Side
Getting desktop pageview data for:  Peter & the Wolf (2006 film)
writing pageview data file for:  Peter & the Wolf (2006 film)
Getting desktop pageview data for:  Once (film)
writing pageview data file 

writing pageview data file for:  Chicago (2002 film)
Getting desktop pageview data for:  The Pianist (2002 film)
writing pageview data file for:  The Pianist (2002 film)
Getting desktop pageview data for:  The Lord of the Rings: The Two Towers
writing pageview data file for:  The Lord of the Rings_ The Two Towers
Getting desktop pageview data for:  Frida (film)
writing pageview data file for:  Frida (film)
Getting desktop pageview data for:  The Hours (film)
writing pageview data file for:  The Hours (film)
Getting desktop pageview data for:  Road to Perdition
writing pageview data file for:  Road to Perdition
Getting desktop pageview data for:  Adaptation (film)
writing pageview data file for:  Adaptation (film)
Getting desktop pageview data for:  Talk to Her
writing pageview data file for:  Talk to Her
Getting desktop pageview data for:  This Charming Man (film)
writing pageview data file for:  This Charming Man (film)
Getting desktop pageview data for:  Spirited Away
writing pagevie

...

writing pageview data file for:  The Hole (1962 film)
Getting desktop pageview data for:  Holiday Inn (film)
writing pageview data file for:  Holiday Inn (film)
Getting desktop pageview data for:  The Horse with the Flying Tail
writing pageview data file for:  The Horse with the Flying Tail
Getting desktop pageview data for:  The House I Live In (1945 film)
writing pageview data file for:  The House I Live In (1945 film)
Getting desktop pageview data for:  The House on 92nd Street
writing pageview data file for:  The House on 92nd Street
Getting desktop pageview data for:  How Green Was My Valley (film)
writing pageview data file for:  How Green Was My Valley (film)
Getting desktop pageview data for:  How the West Was Won (film)
writing pageview data file for:  How the West Was Won (film)
Getting desktop pageview data for:  How to Sleep
writing pageview data file for:  How to Sleep
Getting desktop pageview data for:  Hud (1963 film)
writing pageview data file for:  Hud (1963 film)
Gett

writing pageview data file for:  Main Street on the March!
Getting desktop pageview data for:  The Man Who Knew Too Much (1956 film)
writing pageview data file for:  The Man Who Knew Too Much (1956 film)
Getting desktop pageview data for:  Manhattan Melodrama
writing pageview data file for:  Manhattan Melodrama
Getting desktop pageview data for:  Marie-Louise (film)
writing pageview data file for:  Marie-Louise (film)
Getting desktop pageview data for:  Marty (film)
writing pageview data file for:  Marty (film)
Getting desktop pageview data for:  Mary Poppins (film)
writing pageview data file for:  Mary Poppins (film)
Getting desktop pageview data for:  Men Against the Arctic
writing pageview data file for:  Men Against the Arctic
Getting desktop pageview data for:  The Merry Widow (1934 film)
writing pageview data file for:  The Merry Widow (1934 film)
Getting desktop pageview data for:  A Midsummer Night's Dream (1935 film)
writing pageview data file for:  A Midsummer Night's Dream (

writing pageview data file for:  Pollyanna (1960 film)
Getting desktop pageview data for:  Porgy and Bess (film)
writing pageview data file for:  Porgy and Bess (film)
Getting desktop pageview data for:  Portrait of Jennie
writing pageview data file for:  Portrait of Jennie
Getting desktop pageview data for:  Prelude to War
writing pageview data file for:  Prelude to War
Getting desktop pageview data for:  Pride and Prejudice (1940 film)
writing pageview data file for:  Pride and Prejudice (1940 film)
Getting desktop pageview data for:  The Pride of the Yankees
writing pageview data file for:  The Pride of the Yankees
Getting desktop pageview data for:  Princess O'Rourke
writing pageview data file for:  Princess O'Rourke
Getting desktop pageview data for:  The Private Life of Henry VIII
writing pageview data file for:  The Private Life of Henry VIII
Getting desktop pageview data for:  The Private Life of the Gannets
writing pageview data file for:  The Private Life of the Gannets
Getti

writing pageview data file for:  Stagecoach (1939 film)
Getting desktop pageview data for:  Stairway to Light
writing pageview data file for:  Stairway to Light
Getting desktop pageview data for:  Stalag 17
writing pageview data file for:  Stalag 17
Getting desktop pageview data for:  Star in the Night
writing pageview data file for:  Star in the Night
Getting desktop pageview data for:  A Star Is Born (1937 film)
writing pageview data file for:  A Star Is Born (1937 film)
Getting desktop pageview data for:  State Fair (1945 film)
writing pageview data file for:  State Fair (1945 film)
Getting desktop pageview data for:  The Story of Louis Pasteur
writing pageview data file for:  The Story of Louis Pasteur
Getting desktop pageview data for:  The Stratton Story
writing pageview data file for:  The Stratton Story
Getting desktop pageview data for:  Street Angel (1928 film)
writing pageview data file for:  Street Angel (1928 film)
Getting desktop pageview data for:  A Streetcar Named Desi

writing pageview data file for:  The Vanishing Prairie
Getting desktop pageview data for:  The Virgin Spring
writing pageview data file for:  The Virgin Spring
Getting desktop pageview data for:  Viva Villa!
writing pageview data file for:  Viva Villa!
Getting desktop pageview data for:  Viva Zapata!
writing pageview data file for:  Viva Zapata!
Getting desktop pageview data for:  Waikiki Wedding
writing pageview data file for:  Waikiki Wedding
Getting desktop pageview data for:  The Walls of Malapaga
writing pageview data file for:  The Walls of Malapaga
Getting desktop pageview data for:  The War of the Worlds (1953 film)
writing pageview data file for:  The War of the Worlds (1953 film)
Getting desktop pageview data for:  Watch on the Rhine
writing pageview data file for:  Watch on the Rhine
Getting desktop pageview data for:  Water Birds
writing pageview data file for:  Water Birds
Getting desktop pageview data for:  The Way of All Flesh (1927 film)
writing pageview data file for: 

In [9]:
get_pageView_data_with_template(ARTICLE_PAGEVIEWS_PARAMS_MOBILE_APP, "raw_data", article_list)

Getting mobile-app pageview data for:  Everything Everywhere All at Once
writing pageview data file for:  Everything Everywhere All at Once
Getting mobile-app pageview data for:  All Quiet on the Western Front (2022 film)
writing pageview data file for:  All Quiet on the Western Front (2022 film)
Getting mobile-app pageview data for:  The Whale (2022 film)
writing pageview data file for:  The Whale (2022 film)
Getting mobile-app pageview data for:  Top Gun: Maverick
writing pageview data file for:  Top Gun_ Maverick
Getting mobile-app pageview data for:  Black Panther: Wakanda Forever
writing pageview data file for:  Black Panther_ Wakanda Forever
Getting mobile-app pageview data for:  Avatar: The Way of Water
writing pageview data file for:  Avatar_ The Way of Water
Getting mobile-app pageview data for:  Women Talking (film)
writing pageview data file for:  Women Talking (film)
Getting mobile-app pageview data for:  Guillermo del Toro's Pinocchio
writing pageview data file for:  Guill

writing pageview data file for:  Spider-Man_ Into the Spider-Verse
Getting mobile-app pageview data for:  The Shape of Water
writing pageview data file for:  The Shape of Water
Getting mobile-app pageview data for:  Dunkirk (2017 film)
writing pageview data file for:  Dunkirk (2017 film)
Getting mobile-app pageview data for:  Three Billboards Outside Ebbing, Missouri
writing pageview data file for:  Three Billboards Outside Ebbing, Missouri
Getting mobile-app pageview data for:  Darkest Hour (film)
writing pageview data file for:  Darkest Hour (film)
Getting mobile-app pageview data for:  Blade Runner 2049
writing pageview data file for:  Blade Runner 2049
Getting mobile-app pageview data for:  Coco (2017 film)
writing pageview data file for:  Coco (2017 film)
Getting mobile-app pageview data for:  Phantom Thread
writing pageview data file for:  Phantom Thread
Getting mobile-app pageview data for:  Call Me by Your Name (film)
writing pageview data file for:  Call Me by Your Name (film)

Getting mobile-app pageview data for:  The Great Beauty
writing pageview data file for:  The Great Beauty
Getting mobile-app pageview data for:  20 Feet from Stardom
writing pageview data file for:  20 Feet from Stardom
Getting mobile-app pageview data for:  Argo (2012 film)
writing pageview data file for:  Argo (2012 film)
Getting mobile-app pageview data for:  Life of Pi (film)
writing pageview data file for:  Life of Pi (film)
Getting mobile-app pageview data for:  Les Misérables (2012 film)
writing pageview data file for:  Les Misérables (2012 film)
Getting mobile-app pageview data for:  Lincoln (film)
writing pageview data file for:  Lincoln (film)
Getting mobile-app pageview data for:  Django Unchained
writing pageview data file for:  Django Unchained
Getting mobile-app pageview data for:  Skyfall
writing pageview data file for:  Skyfall
Getting mobile-app pageview data for:  Silver Linings Playbook
writing pageview data file for:  Silver Linings Playbook
Getting mobile-app pagev

writing pageview data file for:  There Will Be Blood
Getting mobile-app pageview data for:  La Vie en rose (film)
writing pageview data file for:  La Vie en rose (film)
Getting mobile-app pageview data for:  Atonement (2007 film)
writing pageview data file for:  Atonement (2007 film)
Getting mobile-app pageview data for:  Michael Clayton (film)
writing pageview data file for:  Michael Clayton (film)
Getting mobile-app pageview data for:  Ratatouille (film)
writing pageview data file for:  Ratatouille (film)
Getting mobile-app pageview data for:  Juno (film)
writing pageview data file for:  Juno (film)
Getting mobile-app pageview data for:  Sweeney Todd: The Demon Barber of Fleet Street (2007 film)
writing pageview data file for:  Sweeney Todd_ The Demon Barber of Fleet Street (2007 film)
Getting mobile-app pageview data for:  The Golden Compass (film)
writing pageview data file for:  The Golden Compass (film)
Getting mobile-app pageview data for:  Elizabeth: The Golden Age
writing page

writing pageview data file for:  Two Soldiers (2003 film)
Getting mobile-app pageview data for:  Monster (2003 film)
writing pageview data file for:  Monster (2003 film)
Getting mobile-app pageview data for:  Harvie Krumpet
writing pageview data file for:  Harvie Krumpet
Getting mobile-app pageview data for:  Chernobyl Heart
writing pageview data file for:  Chernobyl Heart
Getting mobile-app pageview data for:  The Fog of War
writing pageview data file for:  The Fog of War
Getting mobile-app pageview data for:  Chicago (2002 film)
writing pageview data file for:  Chicago (2002 film)
Getting mobile-app pageview data for:  The Pianist (2002 film)
writing pageview data file for:  The Pianist (2002 film)
Getting mobile-app pageview data for:  The Lord of the Rings: The Two Towers
writing pageview data file for:  The Lord of the Rings_ The Two Towers
Getting mobile-app pageview data for:  Frida (film)
writing pageview data file for:  Frida (film)
Getting mobile-app pageview data for:  The H

...

writing pageview data file for:  Lady Be Good (1941 film)
Getting mobile-app pageview data for:  The Last Command (1928 film)
writing pageview data file for:  The Last Command (1928 film)
Getting mobile-app pageview data for:  Laura (1944 film)
writing pageview data file for:  Laura (1944 film)
Getting mobile-app pageview data for:  The Lavender Hill Mob
writing pageview data file for:  The Lavender Hill Mob
Getting mobile-app pageview data for:  Lawrence of Arabia (film)
writing pageview data file for:  Lawrence of Arabia (film)
Getting mobile-app pageview data for:  Leave Her to Heaven
writing pageview data file for:  Leave Her to Heaven
Getting mobile-app pageview data for:  Lend a Paw
writing pageview data file for:  Lend a Paw
Getting mobile-app pageview data for:  Les Girls
writing pageview data file for:  Les Girls
Getting mobile-app pageview data for:  A Letter to Three Wives
writing pageview data file for:  A Letter to Three Wives
Getting mobile-app pageview data for:  The Lif

writing pageview data file for:  North West Mounted Police (film)
Getting mobile-app pageview data for:  Now, Voyager
writing pageview data file for:  Now, Voyager
Getting mobile-app pageview data for:  An Occurrence at Owl Creek Bridge (film)
writing pageview data file for:  An Occurrence at Owl Creek Bridge (film)
Getting mobile-app pageview data for:  Of Pups and Puzzles
writing pageview data file for:  Of Pups and Puzzles
Getting mobile-app pageview data for:  Oklahoma! (1955 film)
writing pageview data file for:  Oklahoma! (1955 film)
Getting mobile-app pageview data for:  The Old Man and the Sea (1958 film)
writing pageview data file for:  The Old Man and the Sea (1958 film)
Getting mobile-app pageview data for:  The Old Mill
writing pageview data file for:  The Old Mill
Getting mobile-app pageview data for:  On the Town (film)
writing pageview data file for:  On the Town (film)
Getting mobile-app pageview data for:  On the Waterfront
writing pageview data file for:  On the Water

writing pageview data file for:  Seven Days to Noon
Getting mobile-app pageview data for:  The Seventh Veil
writing pageview data file for:  The Seventh Veil
Getting mobile-app pageview data for:  Shane (film)
writing pageview data file for:  Shane (film)
Getting mobile-app pageview data for:  Shanghai Express (film)
writing pageview data file for:  Shanghai Express (film)
Getting mobile-app pageview data for:  She Wore a Yellow Ribbon
writing pageview data file for:  She Wore a Yellow Ribbon
Getting mobile-app pageview data for:  Shoeshine (film)
writing pageview data file for:  Shoeshine (film)
Getting mobile-app pageview data for:  The Silent World
writing pageview data file for:  The Silent World
Getting mobile-app pageview data for:  The Sin of Madelon Claudet
writing pageview data file for:  The Sin of Madelon Claudet
Getting mobile-app pageview data for:  Since You Went Away
writing pageview data file for:  Since You Went Away
Getting mobile-app pageview data for:  Skippy (film)

writing pageview data file for:  Thunderball (film)
Getting mobile-app pageview data for:  Thursday's Children
writing pageview data file for:  Thursday's Children
Getting mobile-app pageview data for:  The Time Machine (1960 film)
writing pageview data file for:  The Time Machine (1960 film)
Getting mobile-app pageview data for:  A Time Out of War
writing pageview data file for:  A Time Out of War
Getting mobile-app pageview data for:  Tin Pan Alley (film)
writing pageview data file for:  Tin Pan Alley (film)
Getting mobile-app pageview data for:  The Titan: Story of Michelangelo
writing pageview data file for:  The Titan_ Story of Michelangelo
Getting mobile-app pageview data for:  Titanic (1953 film)
writing pageview data file for:  Titanic (1953 film)
Getting mobile-app pageview data for:  To Catch a Thief
writing pageview data file for:  To Catch a Thief
Getting mobile-app pageview data for:  To Each His Own (1946 film)
writing pageview data file for:  To Each His Own (1946 film)


writing pageview data file for:  The Yearling (1946 film)
Getting mobile-app pageview data for:  Yesterday, Today and Tomorrow
writing pageview data file for:  Yesterday, Today and Tomorrow
Getting mobile-app pageview data for:  You Can't Take It with You (film)
writing pageview data file for:  You Can't Take It with You (film)
Getting mobile-app pageview data for:  Zorba the Greek (film)
writing pageview data file for:  Zorba the Greek (film)


Above output should show dictionaries with views per month